# Classification 

- scegliere classificatori
- prova bilanciamento classi
- prova incremento performance con metadati

class da provare:
- Gaussian Naive Bayes, Decision Tree, K Nearest Neighbors, Random Forest, SVM, logistic regression



Ci sono diversi metodi di estrazione delle feature dal testo, o rappresentazione del testo, che convertono il testo in una forma numerica comprensibile dai modelli di machine learning. Alcuni dei metodi più comuni includono:

1. **Bag of Words (BoW):**
   - **Descrizione:** Questo è uno dei metodi più semplici. Si crea un "sacchetto di parole" che contiene tutte le parole uniche nel corpus di testo. La rappresentazione di ciascun documento è quindi un vettore che conta quante volte ciascuna parola appare nel documento.
   - **Librerie:** `CountVectorizer` di scikit-learn.

2. **Term Frequency-Inverse Document Frequency (TF-IDF):**
   - **Descrizione:** Questo metodo assegna un peso a ciascuna parola in base alla sua frequenza nel documento (Term Frequency) e alla sua rarità nell'intero corpus (Inverse Document Frequency). In questo modo, le parole comuni otterranno pesi più bassi rispetto a quelle più rare.
   - **Librerie:** `TfidfVectorizer` di scikit-learn.

3. **Word Embeddings (Word2Vec, GloVe, FastText):**
   - **Descrizione:** Questi modelli apprendono rappresentazioni vettoriali densi delle parole attraverso il contesto in cui compaiono. Ciascuna parola è mappata in uno spazio vettoriale continuo, e la somiglianza semantica tra le parole viene catturata dalla distanza in questo spazio.
   - **Librerie:** `gensim` per Word2Vec, `spaCy` per GloVe, e `fastText` per FastText.

4. **Doc2Vec:**
   - **Descrizione:** Una estensione di Word2Vec che estende la rappresentazione vettoriale alle frasi o ai documenti interi, assegnando un vettore unico a ciascun documento.
   - **Librerie:** `gensim` fornisce un'implementazione di Doc2Vec.

5. **N-grams:**
   - **Descrizione:** Questo metodo considera sequenze contigue di N parole come feature anziché singole parole. Ad esempio, per N=2, considererai coppie di parole (bigrammi) come feature.
   - **Librerie:** `CountVectorizer` di scikit-learn supporta l'estrazione di n-grammi.

6. **Hashing Vectorizer:**
   - **Descrizione:** Questo è un metodo di hashing che converte le parole in feature utilizzando una funzione hash. Aiuta a ridurre la dimensionalità dello spazio delle feature.
   - **Librerie:** `HashingVectorizer` di scikit-learn.

7. **Embedding Pre-trainati:**
   - **Descrizione:** Puoi utilizzare modelli di embedding pre-addestrati su grandi corpus di testo (come Word2Vec pre-addestrato su Google News) e utilizzare queste rappresentazioni come feature per i tuoi dati.
   - **Librerie:** `gensim`, `spaCy`, e altri per l'uso di modelli pre-addestrati.

La scelta del metodo dipende spesso dal tipo di dati, dalla dimensione del dataset, e dagli obiettivi specifici del tuo task di machine learning. Esperimenti con diversi approcci possono aiutarti a determinare quale funziona meglio per il tuo caso specifico.

**provare classificatori e poi vedere se i metadati aiutano ad aumentare accuracy**


In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold, cross_validate, cross_val_score, cross_val_predict, train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, learning_curve
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, roc_auc_score, make_scorer
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, PrecisionRecallDisplay

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


In [24]:
df = pd.read_csv("df_preprocessed.csv")
df.head()

,tweet_id,tweet,is_hate_speech,dataset,created_at,retweet_count,favorite_count,source,is_reply,is_retweet,is_quote,anonymized_user_id,user_created_at,statuses_count,followers_count,friends_count,anonymized_description,Weighted_Engagement,lunghezza_tweet
0,217874450618134,con tutte le denunce che si sta beccando salv...,1,politics,2018-08-11,0.0,6.0,"<a href=""http://twitter.com/download/iphone"" r...",1.0,0.0,0.0,8.868893e+14,2018-04-01,554.0,748.0,753.0,Avete presente quegli stereotipi sui siciliani...,1.9,120
1,360042217507605,prescrizione i tre magi hanno trovato laccord...,0,politics,2018-11-08,154.0,448.0,"<a href=""http://twitter.com/download/iphone"" r...",0.0,0.0,0.0,2.570741e+14,2011-10-26,35043.0,10838.0,4535.0,Prof di latino e Deputata PD.Ama lo sport la R...,211.4,252
2,817917484817935,il m5s ha votato contro le unionicivili adduce...,1,politics,2018-11-10,195.0,638.0,"<a href=""http://twitter.com/download/iphone"" r...",0.0,0.0,0.0,6.908473e+14,2009-09-24,139750.0,7971.0,1866.0,"Medico, appassionato di Cinema d'Autore, Music...",288.9,255
3,172580609652325,la lega e il m5s stanno dando a bere allopinio...,0,politics,2018-11-22,112.0,377.0,"<a href=""http://twitter.com/download/android"" ...",0.0,0.0,0.0,9.027930e+14,2012-11-27,3356.0,8493.0,5303.0,NaN,169.1,244
4,145836038456701,che cosa cambia questa legge caro con il decr...,0,politics,2018-12-02,45.0,143.0,"<a href=""http://twitter.com/#!/download/ipad"" ...",0.0,0.0,0.0,5.308388e+14,2014-01-14,11234.0,10815.0,1810.0,Veneto! Assessore Regionale Sviluppo Economico...,65.4,193


Data preprocessing

Applying word embedding with the help of SKlearn's fucntion 'TFidfVectorizer', which turns a collection a documents, in this case, news articles, into a sparse matrix containing the vectors that represent the words in the document and their context and meaning


**TfidfVectorizer e CountVectorizer**

In [25]:
X = df['tweet']
y = df['is_hate_speech']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.3, random_state=0)

TFvectorizer = TfidfVectorizer(stop_words='english',
                               ngram_range=(1,2),
                               max_df=.8,
                               min_df=.0001
                               )

X_train = TFvectorizer.fit_transform(X_train)
X_test = TFvectorizer.transform(X_test)

# # In case CountVectorizer is preferred
# Cntvectorizer = CountVectorizer(stop_words='english',
#                                 ngram_range=(1,2),
#                                 max_df=.8)

# X_train = Cntvectorizer.fit_transform(X_train)
# X_test = Cntvectorizer.transform(X_test)

# Looking at the distribution of labels in the target variable, as well as info on the X_train matrix
print(np.unique(y_train, return_counts=True))
X_train

(array([0, 1], dtype=int64), array([4668, 2332], dtype=int64))


<7000x135931 sparse matrix of type '<class 'numpy.float64'>'
	with 297703 stored elements in Compressed Sparse Row format>

Quick-testing a model, to check if the vectorizing phase went well and a classifying model is able to correctly detect fake news. For this, a simple model with the default parameters is used.

Aditionally, a classification report is produced together with a confusion matrix, a ROC curve and a Precision-Recall curve to further analyze the performance of the classifying task.

In [5]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits = 4))
#da usare ConfusionMatrixDisplay

0.733
              precision    recall  f1-score   support

           0     0.7691    0.8571    0.8107      2001
           1     0.6286    0.4845    0.5472       999

    accuracy                         0.7330      3000
   macro avg     0.6988    0.6708    0.6789      3000
weighted avg     0.7223    0.7330    0.7229      3000



Testing several different models with cross-validation, to see if other models perform better regarding false negatives, and thus, the recall score. To evaluate them, a 10-fold cross validation approach is used to further avoid the unwanted introudction of bias to the model, expecting that the model fits the data well enough.

After this evaluation, the precision recall curve, along with a plot to visualize their different metrics is produced.


In [6]:
# Initializing the models to test
model_dt = DecisionTreeClassifier()
model_logreg = LogisticRegression()
model_linsvc = LinearSVC()
model_knn = KNeighborsClassifier()
model_paa = PassiveAggressiveClassifier()

# Creating a dictionary with the models and their name to feed to the cross-validation evaluation
models = {
    'Decision Tree': model_dt,
    'Logistic Regression': model_logreg,
    'Linear SVM': model_linsvc,
    'K-NearestNeighbors': model_knn,
    'PassiveAgressive': model_paa
    }

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
acc = {}
prec = {}
rec = {}
f1 = {}

# Metrics to retrieve after the cross-validation is performed
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

for name, model in models.items():
    print(f"{name} results:")
    score = cross_validate(model, X_train, y_train, scoring=scoring, cv=kf, n_jobs=-1, verbose=0)
    print("Accuracy: ", round(score['test_accuracy'].mean(),4))
    print("Precision: ", round(score['test_precision'].mean(),4))
    print("Recall: ", round(score['test_recall'].mean(),4))
    print("F1_score: ", round(score['test_f1_score'].mean(),4), '\n')

    acc[name] = score['test_accuracy'].mean()
    prec[name] = score['test_precision'].mean()
    rec[name] = score['test_recall'].mean()
    f1[name] = score['test_f1_score'].mean()

Decision Tree results:
Accuracy:  0.7721
Precision:  0.6801
Recall:  0.5945
F1_score:  0.6339 

Logistic Regression results:
Accuracy:  0.7501
Precision:  0.9663
Recall:  0.259
F1_score:  0.4079 

Linear SVM results:
Accuracy:  0.8064
Precision:  0.9104
Recall:  0.4644
F1_score:  0.6141 

K-NearestNeighbors results:
Accuracy:  0.7346
Precision:  0.6359
Recall:  0.4672
F1_score:  0.5377 

PassiveAgressive results:
Accuracy:  0.8156
Precision:  0.8591
Recall:  0.5332
F1_score:  0.6568 



In [16]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay

In [35]:
# Plotting the PR curves for every model
model_dt.fit(X_train, y_train), model_logreg.fit(X_train, y_train), model_linsvc.fit(X_train, y_train), model_knn.fit(X_train, y_train), model_paa.fit(X_train, y_train)

plot_precision_recall_curve(model_dt, X_test, y_test, ax = plt.gca(),name = "Decision Tree")
plot_precision_recall_curve(model_logreg, X_test, y_test, ax = plt.gca(),name = "Log Regression")
plot_precision_recall_curve(model_linsvc, X_test, y_test, ax = plt.gca(),name = "Linear SVM")
plot_precision_recall_curve(model_knn, X_test, y_test, ax = plt.gca(),name = "kNN")
plot_precision_recall_curve(model_paa, X_test, y_test, ax = plt.gca(),name = "PAS")
#plt.savefig('PR-Curve.png', dpi=600) 
plt.title('Precision-Recall curve')
